# 準備

In [8]:
#pip install langchain_openai

In [2]:
import getpass
import os

# OpenAI API キーの設定
api_key = getpass.getpass("OpenAI API キーを入力してください: ")
os.environ["OPENAI_API_KEY"] = api_key

OpenAI API キーを入力してください:  ········


# チャットアプリ

In [10]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage

# 1 モデルの定義
llm = ChatOpenAI(model="gpt-4o-mini")

history = []
n = 10
for i in range(10):
    user_input = input("ユーザ入力: ")
    if user_input == "exit":
        break
    # 2 HumanMessage の作成と表示
    human_message = HumanMessage(user_input)
    human_message.pretty_print()
    # 3 会話履歴の追加
    history.append(HumanMessage(user_input))
    # 4 応答の作成と表示
    ai_message = llm.invoke(history)
    ai_message.pretty_print()
    # 5 会話履歴の追加
    history.append(ai_message)

ユーザ入力:  こんばんは


================================ Human Message =================================

こんばんは
================================== Ai Message ==================================

こんばんは！今日はどのようなことをお話ししましょうか？


ユーザ入力:  exit


# 翻訳アプリ

In [13]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")

# 1 テンプレートの作成
TRANSLATION_PROMPT = """\
以下の文章を {language} に翻訳し、翻訳結果のみを返してください。
{source_text}
"""
prompt = PromptTemplate.from_template(TRANSLATION_PROMPT)

# 2 Runnable の作成
runnable = prompt | llm

language = "日本語"
source_text = """\
cogito, ergo sum
"""

# 3 Runnable の実行と結果の表示
response = runnable.invoke(dict(language=language, source_text=source_text))
response.pretty_print()

================================== Ai Message ==================================

我思う、ゆえに我あり


# テーブル作成アプリ

In [16]:
from langchain_core.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai.chat_models import ChatOpenAI
import csv


# 1 入力形式の定義
class CSVSaveToolInput(BaseModel):
    filename: str = Field(description="ファイル名")
    csv_text: str = Field(description="CSVのテキスト")


# 2 ツール本体の定義
@tool("csv-save-tool", args_schema=CSVSaveToolInput)
def csv_save(filename: str, csv_text: str) -> bool:
    """CSV テキストをファイルに保存する"""
    # parse CSV text
    try:
        rows = list(csv.reader(csv_text.splitlines()))
    except Exception as e:
        return False

    # save to file
    with open(filename, "w") as f:
        writer = csv.writer(f)
        writer.writerows(rows)

    return True

C:\anaconda\Lib\site-packages\IPython\core\interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


#  ツールを LLM に紐づける

In [19]:
llm = ChatOpenAI(model="gpt-4o-mini")
tools = [csv_save]
llm_with_tool = llm.bind_tools(tools=tools, tool_choice="csv-save-tool")

TABLE_PROMPT = """\
{user_input}

結果は CSV ファイルに保存してください。ただし、ファイル名は上記の内容から適切に決定してください。
"""
prompt = PromptTemplate.from_template(TABLE_PROMPT)
get_tool_args = lambda x: x.tool_calls[0]

# 4 Runnable の作成
runnable = prompt | llm_with_tool | get_tool_args | csv_save

user_input = "フィボナッチ数列の番号と値を10番目まで表にまとめて、CSV ファイルに保存してください。"

# 5 Runnable の実行と結果の確認
response = runnable.invoke(dict(user_input=user_input))
print(response)

content='true' name='csv-save-tool' tool_call_id='call_0etTAJSdsCi5uRYBHeKMMVDW'


# Plan-and-Solve

In [22]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [26]:
from pydantic import BaseModel, Field


# ツール入力形式の定義
class ActionItem(BaseModel):
    action_name: str = Field(description="アクション名")
    action_description: str = Field(description="アクションの詳細")


class Plan(BaseModel):
    """アクションプランを格納する"""

    problem: str = Field(description="問題の説明")
    actions: list[ActionItem] = Field(description="実行すべきアクションリスト")


class ActionResult(BaseModel):
    """実行時の考えと結果を格納する"""

    thoughts: str = Field(description="検討内容")
    result: str = Field(description="結果")

# 単一アクションの実行

In [29]:
from langchain_openai.output_parsers.tools import PydanticToolsParser
from langchain_core.prompts import PromptTemplate


ACTION_PROMPT = """\
問題をアクションプランに分解して解いています。
これまでのアクションの結果と、次に行うべきアクションを示すので、実際にアクションを実行してその結果を報告してください。
# 問題
{problem}
# アクションプラン
{action_items}
# これまでのアクションの結果
{action_results}
# 次のアクション
{next_action}
"""

llm_action = llm.bind_tools([ActionResult], tool_choice="ActionResult")
action_parser = PydanticToolsParser(tools=[ActionResult], first_tool_only=True)
plan_parser = PydanticToolsParser(tools=[Plan], first_tool_only=True)

action_prompt = PromptTemplate.from_template(ACTION_PROMPT)
action_runnable = action_prompt | llm_action | action_parser

# プランに含まれるアクションを実行するループ

In [32]:
from langchain_core.messages import AIMessage


def action_loop(action_plan: Plan):
    problem = action_plan.problem
    actions = action_plan.actions

    action_items = "\n".join(["* " + action.action_name for action in actions])
    action_results = []
    action_results_str = ""
    for i, action in enumerate(actions):
        print("=" * 20)
        print(f"[{i+1}/{len(actions)}]以下のアクションを実行します。")
        print(action.action_name)

        next_action = f"* {action.action_name}  \n{action.action_description}"
        response = action_runnable.invoke(
            dict(
                problem=problem,
                action_items=action_items,
                action_results=action_results_str,
                next_action=next_action,
            )
        )
        action_results.append(response)
        action_results_str += f"* {action.action_name}  \n{response.result}\n"
        print("-" * 10 + "検討内容" + "-" * 10)
        print(response.thoughts)
        print("-" * 10 + "結果" + "-" * 10)
        print(response.result)

    return AIMessage(action_results_str)

# Plan-and-Solve を行うか否かの分岐

In [35]:
def route(ai_message: AIMessage):
    if ai_message.response_metadata["finish_reason"] == "tool_calls":
        return plan_parser | action_loop
    else:
        return ai_message

# 全体を通した Runnable 作成

In [38]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage

PLAN_AND_SOLVE_PROMPT = """\
ユーザの質問が複雑な場合は、アクションプランを作成し、その後に1つずつ実行する Plan-and-Solve 形式をとります。
これが必要と判断した場合は、Plan ツールによってアクションプランを保存してください。
"""
system_prompt = SystemMessage(PLAN_AND_SOLVE_PROMPT)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_prompt, MessagesPlaceholder(variable_name="history")]
)

llm_plan = llm.bind_tools(tools=[Plan])
planning_runnable = chat_prompt | llm_plan | route

# チャット部分の作成

In [41]:
history = []
n = 10
for i in range(10):
    user_input = input("ユーザ入力: ")
    if user_input == "exit":
        break
    # 1 HumanMessage の作成と表示
    human_message = HumanMessage(user_input)
    human_message.pretty_print()
    # 2 会話履歴の追加
    history.append(HumanMessage(user_input))
    # 3 応答の作成と表示
    ai_message = planning_runnable.invoke(dict(history=history))
    ai_message.pretty_print()
    # 4 会話履歴の追加
    history.append(ai_message)

ユーザ入力:  ある製造工場では、1時間に200個の部品が生産されます。工場は1日8時間稼働し、1週間に5日間営業しています。生産された部品のうち5%は品質不良で廃棄されます。この工場では1ヶ月（4週間）に品質不良で廃棄される部品の総数を求めなさい。


================================ Human Message =================================

ある製造工場では、1時間に200個の部品が生産されます。工場は1日8時間稼働し、1週間に5日間営業しています。生産された部品のうち5%は品質不良で廃棄されます。この工場では1ヶ月（4週間）に品質不良で廃棄される部品の総数を求めなさい。
[1/2]以下のアクションを実行します。
計算部品生産量
----------検討内容----------
1時間に200個の部品が生産され、1日8時間、1週間5日間営業なので、月間生産部品数は200個 * 8時間 * 5日 * 4週間で計算することができる。
----------結果----------
月間生産部品数 = 200 * 8 * 5 * 4 = 32000個
[2/2]以下のアクションを実行します。
計算品質不良数
----------検討内容----------
月間生産部品数が32000個で、その5%が品質不良で廃棄されるため、不良品数を算出した。
----------結果----------
品質不良で廃棄される部品数 = 32000 * 0.05 = 1600個
================================== Ai Message ==================================

* 計算部品生産量  
月間生産部品数 = 200 * 8 * 5 * 4 = 32000個
* 計算品質不良数  
品質不良で廃棄される部品数 = 32000 * 0.05 = 1600個


ユーザ入力:  exit
